- Author: Ben Du
- Date: 2020-11-29 21:13:47
- Title: Hands on Dulwich
- Slug: hands-on-Dulwich
- Category: Computer Science
- Tags: Computer Science, programming, Python, Git, Dulwicch, version control
- Modified: 2021-05-29 21:13:47


Note: dulwich is not feature complete yet 
and the development of the project is extremely slow.
It is suggested that you use other Python packages instead.
For more discussions,
please refer to
[Git Implementations and Bindings in Python](http://www.legendu.net/misc/blog/git-implementations-and-bindings-in-python/)
.

## Tips and Traps

1. The `git` command (and thus Dulwich) accepts URLs both with and without the trailing `.git`. 

In [1]:
!pip3 install dulwich

     |████████████████████████████████| 531 kB 3.5 MB/s eta 0:00:01


In [2]:
import dulwich.repo

In [3]:
from dulwich.repo import Repo

In [4]:
import dulwich.porcelain

## git clone

In [5]:
dulwich.porcelain.clone?

Signature:
dulwich.porcelain.clone(
    source,
    target=None,
    bare=False,
    checkout=None,
    errstream=<dulwich.porcelain.NoneStream object at 0x7f38b80b4280>,
    outstream=None,
    origin=b'origin',
    depth=None,
    **kwargs,
)
Docstring:
Clone a local or remote git repository.

Args:
  source: Path or URL for source repository
  target: Path to target repository (optional)
  bare: Whether or not to create a bare repository
  checkout: Whether or not to check-out HEAD after cloning
  errstream: Optional stream to write progress to
  outstream: Optional stream to write progress to (deprecated)
  origin: Name of remote from the repository used to clone
  depth: Depth to fetch at
Returns: The new repository
File:      ~/.local/lib/python3.8/site-packages/dulwich/porcelain.py
Type:      function


In [6]:
url = "https://github.com/dclong/docker-ubuntu_b"
dir_local = "/tmp/test_gitpython"

In [7]:
repo = dulwich.porcelain.clone(
    "https://github.com/dclong/docker-ubuntu_b",
    "/tmp/test_dulwich",
)

In [8]:
repo

<Repo at '/tmp/test_dulwich'>

In [30]:
!ls /tmp/test_dulwich

build.sh  Dockerfile  LICENSE  readme.md  scripts


Clone the local repository to another location 
(which is not very useful as you can directly copy the directory to the new location).

In [25]:
repo.clone?

Signature:
repo.clone(
    target_path,
    mkdir=True,
    bare=False,
    origin=b'origin',
    checkout=None,
)
Docstring:
Clone this repository.

Args:
  target_path: Target path
  mkdir: Create the target directory
  bare: Whether to create a bare repository
  origin: Base name for refs in target repository
    cloned from this repository
Returns: Created repository as `Repo`
File:      ~/.local/lib/python3.8/site-packages/dulwich/repo.py
Type:      method


## dulwich.repo.Repo

In [20]:
type(repo)

dulwich.repo.Repo

In [11]:
repo2 = Repo("/tmp/test_dulwich")
repo2

<Repo at '/tmp/test_dulwich'>

In [21]:
dir(repo)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_graftpoints',
 '_commondir',
 '_controldir',
 '_del_named_file',
 '_determine_file_mode',
 '_get_object',
 '_get_user_identity',
 '_graftpoints',
 '_init_files',
 '_init_maybe_bare',
 '_init_new_working_directory',
 '_put_named_file',
 '_read_heads',
 '_remove_graftpoints',
 '_write_reflog',
 'bare',
 'clone',
 'close',
 'commondir',
 'controldir',
 'create',
 'discover',
 'do_commit',
 'fetch',
 'fetch_objects',
 'fetch_pack_data',
 'generate_pack_data',
 'get_blob_normalizer',
 'get_config',
 'get_config_stack',
 'get_descr

## git add

In [33]:
!touch /tmp/test_dulwich/abc2

In [34]:
!ls /tmp/test_dulwich/

abc  abc2  build.sh  Dockerfile  LICENSE  readme.md  scripts


In [35]:
!git -C /tmp/test_dulwich/ status

On branch dev
Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   abc

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	abc2



In [29]:
dulwich.porcelain.add?

Signature: dulwich.porcelain.add(repo='.', paths=None)
Docstring:
Add files to the staging area.

Args:
  repo: Repository for the files
  paths: Paths to add.  No value passed stages all modified files.
Returns: Tuple with set of added files and ignored files

If the repository contains ignored directories, the returned set will
contain the path to an ignored directory (with trailing slash). Individual
files within ignored directories will not be returned.
File:      ~/.local/lib/python3.8/site-packages/dulwich/porcelain.py
Type:      function


In [32]:
dulwich.porcelain.add("/tmp/test_dulwich", paths="/tmp/test_dulwich/abc")

(['abc'], set())

By default,
dulwich adds all files in the current working directory,
which is not the right behavior!
I have submitted [a ticket](https://github.com/dulwich/dulwich/issues/895) to fix the issue.

In [54]:
dulwich.porcelain.add("/tmp/test_dulwich")

ValueError: '/workdir/archives/blog/misc/content/2020/11/hands-on-dulwich/hands-on-dulwich.ipynb' does not start with '/tmp/test_dulwich'

In [67]:
dulwich.porcelain.add("/tmp/test_dulwich", paths="/tmp/test_dulwich/.")

(['./'], set())

In [71]:
status = dulwich.porcelain.status("/tmp/test_dulwich")
status

GitStatus(staged={'add': [], 'delete': [], 'modify': []}, unstaged=[], untracked=['abc2'])

In [72]:
status.unstaged

[]

In [73]:
status.untracked

['abc2']

## git commit

In [48]:
dulwich.porcelain.commit?

Signature:
dulwich.porcelain.commit(
    repo='.',
    message=None,
    author=None,
    committer=None,
    encoding=None,
    no_verify=False,
)
Docstring:
Create a new commit.

Args:
  repo: Path to repository
  message: Optional commit message
  author: Optional author name and email
  committer: Optional committer name and email
  no_verify: Skip pre-commit and commit-msg hooks
Returns: SHA1 of the new commit
File:      ~/.local/lib/python3.8/site-packages/dulwich/porcelain.py
Type:      function


In [50]:
dulwich.porcelain.commit("/tmp/test_dulwich/", message="add abc")

b'11fb9f18f9d211c93175e898faa731584b8be368'

In [51]:
!git -C /tmp/test_dulwich/ status

On branch dev
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	abc2

nothing added to commit but untracked files present (use "git add" to track)


In [53]:
!git -C /tmp/test_dulwich/ log | head

commit 11fb9f18f9d211c93175e898faa731584b8be368
Author: Benjamin Du <longendu@yahoo.com>
Date:   Sat Aug 7 01:33:45 2021 +0000

    add abc

commit 2fd55f0a653bf8ec2e7ffda16c7b2d601167da06
Author: Ben Chuanlong Du <dclong@users.noreply.github.com>
Date:   Fri Jul 16 09:53:00 2021 -0700



## ConfigFile

ConfigFile inherits ConfigDict which means that you operate on a ConfiFile like a `dict`.

In [14]:
config = repo.get_config()
config

ConfigFile(CaseInsensitiveDict([((b'core',), CaseInsensitiveDict([(b'repositoryformatversion', b'0'), (b'filemode', b'true'), (b'bare', b'false'), (b'logallrefupdates', b'true')])), ((b'remote', b'origin'), CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'), (b'fetch', b'+refs/heads/*:refs/remotes/origin/*')])), ((b'remote', b'nima'), CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'), (b'fetch', b'+refs/heads/*:refs/remotes/nima/*')]))]))

In [37]:
config.keys()

KeysView(ConfigFile(CaseInsensitiveDict([((b'core',), CaseInsensitiveDict([(b'repositoryformatversion', b'0'), (b'filemode', b'true'), (b'bare', b'false'), (b'logallrefupdates', b'true')])), ((b'remote', b'origin'), CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'), (b'fetch', b'+refs/heads/*:refs/remotes/origin/*')])), ((b'remote', b'nima'), CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'), (b'fetch', b'+refs/heads/*:refs/remotes/nima/*')]))])))

In [38]:
config.values()

ValuesView(ConfigFile(CaseInsensitiveDict([((b'core',), CaseInsensitiveDict([(b'repositoryformatversion', b'0'), (b'filemode', b'true'), (b'bare', b'false'), (b'logallrefupdates', b'true')])), ((b'remote', b'origin'), CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'), (b'fetch', b'+refs/heads/*:refs/remotes/origin/*')])), ((b'remote', b'nima'), CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'), (b'fetch', b'+refs/heads/*:refs/remotes/nima/*')]))])))

In [40]:
config[(b"remote", b"origin")]

CaseInsensitiveDict([(b'url', b'https://github.com/dclong/docker-ubuntu_b'),
                     (b'fetch', b'+refs/heads/*:refs/remotes/origin/*')])

In [27]:
config.get((b"remote", b"origin"), b"url")

b'https://github.com/dclong/docker-ubuntu_b'

In [31]:
dict(config)

{(b'core',): CaseInsensitiveDict([(b'repositoryformatversion', b'0'),
                      (b'filemode', b'true'),
                      (b'bare', b'false'),
                      (b'logallrefupdates', b'true')]),
 (b'remote',
  b'origin'): CaseInsensitiveDict([(b'url',
                       b'https://github.com/dclong/docker-ubuntu_b'),
                      (b'fetch', b'+refs/heads/*:refs/remotes/origin/*')]),
 (b'remote',
  b'nima'): CaseInsensitiveDict([(b'url',
                       b'https://github.com/dclong/docker-ubuntu_b'),
                      (b'fetch', b'+refs/heads/*:refs/remotes/nima/*')])}

## git remote -v

In [36]:
[key[1].decode() for key in config.keys() if key[0] == b"remote"]

['origin', 'nima']

## dulwich.porcelain.ls_remote

In [79]:
dulwich.porcelain.ls_remote(url)

{b'HEAD': b'2fd55f0a653bf8ec2e7ffda16c7b2d601167da06',
 b'refs/heads/debian': b'618389f8300615ba7d31c4ba7b75fb94770391e1',
 b'refs/heads/dev': b'2fd55f0a653bf8ec2e7ffda16c7b2d601167da06',
 b'refs/heads/main': b'71f2b1d96cfa8596319686a5a98514ad4ac85506',
 b'refs/pull/1/head': b'b90bd029b4707a94640957cbfae73a631a9d83e0',
 b'refs/pull/1/merge': b'c767a6a929e599c245a4241477093554c1ab0d10',
 b'refs/pull/10/head': b'ea5b60d7fc34fce65ec4a503c20536d3e0d4587a',
 b'refs/pull/11/head': b'0d7e63476077d2ed51728823f3ce54b57a8287e6',
 b'refs/pull/12/head': b'd2fcee067c4f003084950799f8cb408625d8c610',
 b'refs/pull/13/head': b'5d29d66bdadfb7e265b0f895ca1bf6f26f7bad39',
 b'refs/pull/14/head': b'114a4a2af63270ea69f08b5801cf1bdfa1c29222',
 b'refs/pull/15/head': b'ffe1f38e3a91ef69b784b867ad5d7729bf17499f',
 b'refs/pull/16/head': b'1b2ee6dbbe96435009ef96a80776ebc30e74b082',
 b'refs/pull/17/head': b'b1d52163d2170c1b7a8fe9a0d05bf15a7c4dfcb0',
 b'refs/pull/18/head': b'ac8a2c93c8c93985eb7a6e2742e2a3e5aa4786d8',

## dulwich.objects.Commit 

In [43]:
repo.head()

b'2fd55f0a653bf8ec2e7ffda16c7b2d601167da06'

In [45]:
commit = repo[repo.head()]
commit

<Commit b'2fd55f0a653bf8ec2e7ffda16c7b2d601167da06'>

In [46]:
type(commit)

dulwich.objects.Commit

In [39]:
commit.message

b'Update etc.sh'

In [40]:
dir(commit)

['__bytes__',
 '__class__',
 '__cmp__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_author',
 '_author_time',
 '_author_timezone',
 '_author_timezone_neg_utc',
 '_check_has_member',
 '_chunked_text',
 '_commit_time',
 '_commit_timezone',
 '_commit_timezone_neg_utc',
 '_committer',
 '_deserialize',
 '_encoding',
 '_extra',
 '_get_extra',
 '_get_parents',
 '_gpgsig',
 '_header',
 '_is_legacy_object',
 '_mergetag',
 '_message',
 '_needs_serialization',
 '_parents',
 '_parse_file',
 '_parse_legacy_object',
 '_parse_legacy_object_header',
 '_parse_object',
 '_parse_object_header',
 '_serialize',
 '_set_parents',
 '_sha',
 '_tree',
 'as_legacy_object',
 'as_legacy_object_chunks',
 'as_pretty_strin

In [41]:
main = heads.main
main

NameError: name 'heads' is not defined

Get the commit pointed to by head called master.

In [17]:
main.commit

<git.Commit "95ed236bd715a06320ee85d519fb79a0adffe072">

In [18]:
main.rename("main2")

<git.Head "refs/heads/main2">

Verify that the `main` branch has been renamed to `main2`.

In [19]:
!cd {dir_local} && git branch

* main2


### Get the Active Branch

In [5]:
repo.active_branch.name

'main'

## Get All Branches

In [10]:
repo.branches

[<git.Head "refs/heads/main">]

### Get the Remote Name

In [6]:
repo.remote().name

'origin'

### Get all Remotes

In [7]:
repo.remotes

[<git.Remote "origin">]

## Commits

Get the latest commit in a branch.

In [23]:
repo.commit("main")

<git.Commit "53d99955a9762427f2f68dc04765471089055dc1">

In [28]:
repo.commit("main").diff(repo.commit("origin/dev"))

[]

In [27]:
repo.commit("origin/dev")

<git.Commit "8f9f426f13d70b21f573f7c50bbe01e8ce38f158">

In [26]:
repo.refs

[<git.Head "refs/heads/main">,
 <git.RemoteReference "refs/remotes/origin/HEAD">,
 <git.RemoteReference "refs/remotes/origin/debian">,
 <git.RemoteReference "refs/remotes/origin/dev">,
 <git.RemoteReference "refs/remotes/origin/main">]

## Changed Files

Update a file.

In [23]:
!echo "# add a line of comment" >> {dir_local}/build.sh

In [24]:
repo = Repo(dir_local)
files_changed = [item.a_path for item in repo.index.diff(None)]
files_changed

['build.sh']

## Staged Files

In [25]:
repo = Repo(dir_local)
index = repo.index

In [26]:
index.add("build.sh")

[(100644, f1cb16a21febd1f69a7a638402dddeb7f1dc9771, 0, build.sh)]

The file `build.sh` is now staged.

In [27]:
files_stage = [item.a_path for item in repo.index.diff('HEAD')]
files_stage

['build.sh']

In [28]:
files_changed = [item.a_path for item in repo.index.diff(None)]
files_changed

[]

Commit the change.

In [29]:
index.commit("update build.sh")

<git.Commit "bfea304786b7b77f7fe247c74040c0e23576fc41">

In [30]:
files_stage = [item.a_path for item in repo.index.diff('HEAD')]
files_stage

[]

In [8]:
remote = repo.remote()
remote

<git.Remote "origin">

## Push the Commits

Push the local `main2` branch to the remote `main2` branch.

In [32]:
remote.push("main2")

The above is equivalent to the following more detailed specification.

In [62]:
remote.push("refs/heads/main2:refs/heads/main2")

Push the local `main2` branch to the remote `main` branch.

In [63]:
remote.push("refs/heads/main2:refs/heads/main")

## Pull a Branch

In [6]:
repo.active_branch

<git.Head "refs/heads/main">

In [11]:
remote.pull(repo.active_branch)

[]

In [12]:
!ls {dir_local}

abc       build.sh  readme.md


## git checkout

In [42]:
help(repo.refs[4].checkout)

Help on method checkout in module git.refs.head:

checkout(force=False, **kwargs) method of git.refs.remote.RemoteReference instance
    Checkout this head by setting the HEAD to this reference, by updating the index
    to reflect the tree we point to and by updating the working tree to reflect
    the latest index.
    
    The command will fail if changed working tree files would be overwritten.
    
    :param force:
        If True, changes to the index and the working tree will be discarded.
        If False, GitCommandError will be raised in that situation.
    
    :param kwargs:
        Additional keyword arguments to be passed to git checkout, i.e.
        b='new_branch' to create a new branch at the given spot.
    
    :return:
        The active branch after the checkout operation, usually self unless
        a new branch has been created.
        If there is no active branch, as the HEAD is now detached, the HEAD
        reference will be returned instead.
    
    :note:

## git tag

List all tags.

In [13]:
repo.tags

[]

Add a tag.

In [15]:
repo.create_tag("v1.0.0")

<git.TagReference "refs/tags/v1.0.0">

In [17]:
repo.tags

[<git.TagReference "refs/tags/v1.0.0">]

In [20]:
repo.tag("refs/tags/v1.0.0")

<git.TagReference "refs/tags/v1.0.0">

In [23]:
tag2 = repo.tag("refs/tags/v2.0.0")
tag2

<git.TagReference "refs/tags/v2.0.0">

In [24]:
repo.tags

[<git.TagReference "refs/tags/v1.0.0">]

The GitCommandError is thrown when the tag already exists.

In [25]:
repo.create_tag("v1.0.0")

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git tag v1.0.0 HEAD
  stderr: 'fatal: tag 'v1.0.0' already exists'

In [26]:
repo.remote().push("v1.0.0")

## git diff

In [16]:
help(repo.refs[4].commit.diff)

Help on method diff in module git.diff:

diff(other: Union[Type[git.diff.Diffable.Index], Type[ForwardRef('Tree')], object, NoneType, str] = <class 'git.diff.Diffable.Index'>, paths: Union[str, List[str], Tuple[str, ...], NoneType] = None, create_patch: bool = False, **kwargs: Any) -> 'DiffIndex' method of git.objects.commit.Commit instance
    Creates diffs between two items being trees, trees and index or an
    index and the working tree. It will detect renames automatically.
    
    :param other:
        Is the item to compare us with.
        If None, we will be compared to the working tree.
        If Treeish, it will be compared against the respective tree
        If Index ( type ), it will be compared against the index.
        If git.NULL_TREE, it will compare against the empty tree.
        It defaults to Index to assure the method will not by-default fail
        on bare repositories.
    
    :param paths:
        is a list of paths or a single path to limit the diff to.
 

In [3]:
url = "https://github.com/dclong/docker-ubuntu_b.git"
dir_local = "/tmp/" + url[(url.rindex("/") + 1):]
!rm -rf {dir_local}

In [4]:
repo = git.Repo.clone_from(url, dir_local, branch="main")
repo

<git.repo.base.Repo '/tmp/docker-ubuntu_b.git/.git'>

In [25]:
repo.refs

[<git.Head "refs/heads/debian">,
 <git.Head "refs/heads/dev">,
 <git.Head "refs/heads/main">,
 <git.RemoteReference "refs/remotes/origin/HEAD">,
 <git.RemoteReference "refs/remotes/origin/debian">,
 <git.RemoteReference "refs/remotes/origin/dev">,
 <git.RemoteReference "refs/remotes/origin/main">]

In [6]:
diffs = repo.refs[4].commit.diff(repo.refs[3].commit)
diffs

[]

In [21]:
diffs = repo.refs[4].commit.diff(repo.refs[2].commit)
diffs

[<git.diff.Diff at 0x7f0eb05d1a60>, <git.diff.Diff at 0x7f0eb05d1af0>]

In [13]:
str(diffs[0])

'Dockerfile\n=======================================================\nlhs: 100644 | 8ae5c7650a8c031a8e176d896a3665bbe7e2aae8\nrhs: 100644 | 9f2304d9a97aa1279ad1938b3bb74790172c9d8b'

In [12]:
repo.refs[5].name

'origin/main'

In [6]:
print(repo.git.status())

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [6]:
repo.git.checkout("debian", force=True)

"Your branch is up to date with 'origin/debian'."

In [8]:
repo.git.checkout(b="a_new_branch", force=True)

''

In [ ]:
nima = repo.refs[4].checkout(force=True, b="nima")
nima

In [50]:
diffs = nima.commit.diff(repo.refs[-1].commit)
diffs[0].diff

''

Diff the `dev` and the `main` branch,
which is equivalent to the Git command 
`git diff dev..main`.

In [30]:
repo.refs[2].commit.diff(repo.refs[1].commit)

[]

In [32]:
diffs = repo.refs[2].commit.diff(repo.refs[0].commit)
diffs

In [33]:
diffs[0]

In [24]:
diffs = repo.refs[6].commit.diff(repo.refs[7].commit)
diffs

[]

In [25]:
diffs = repo.refs[4].commit.diff(repo.refs[7].commit)
diffs

In [26]:
diffs[0].diff

''

In [27]:
diffs = repo.refs[7].commit.diff(repo.refs[4].commit)
diffs

In [28]:
diffs[0].diff

''

In [19]:
any(ele for ele in [''])

False

In [23]:
repo.branches[0].name

'dev'

In [ ]:
for branch in repo.branches:
    branch.

In [12]:
commit = repo.head.commit
commit

<git.Commit "6716bb0d016bd63ba543f3d9c67a65dadecd152e">

In [15]:
type(repo.branches[0])

git.refs.head.Head

In [17]:
repo.refs[4].commit.diff(repo.refs[2].commit)

[]

In [9]:
repo.refs[4].commit.diff(repo.refs[3].commit)

In [20]:
help(repo.git.branch)

Help on function <lambda> in module git.cmd:

<lambda> lambda *args, **kwargs



In [28]:
repo.heads

[<git.Head "refs/heads/dev">, <git.Head "refs/heads/main">]

Diff the `debian` and the `main` branches but limit diff to specified paths 
(via the `paths` parameter).

In [24]:
diffs = repo.refs[4].commit.diff(repo.refs[2].commit, paths=["build.sh", "scripts"])
diffs

[]

## References

https://github.com/dulwich/dulwich

https://www.dulwich.io/docs/api/